# Training Experiments

In [1]:
import h2oai_client
import numpy as np
import pandas as pd
# import h2o
import requests
import math
from h2oai_client import Client, ModelParameters, InterpretParameters

In [2]:
ip = 54.164.160.82
address = 'http://' + ip + ':12345'
username = 'training'
password = 'training'

h2oai = Client(address = address
               , username = username
               , password = password)

## Boston Housing

In [7]:
dataPath = '/data/Training/BostonHousing.csv'
basename = 'Housing'
target = 'VALUE'
ratio = 0.8

boston_data = h2oai.create_dataset_sync(dataPath)

# Split the data
boston_split_data = h2oai.make_dataset_split(
    dataset_key = boston_data.key
    , output_name1 = basename + "_train"
    , output_name2 = basename + "_test"
    , target = target
    , fold_col = ""
    , time_col = ""
    , ratio = ratio
    , seed = 1234
)

train_key = h2oai.get_dataset_split_job(boston_split_data).entity[0]
test_key  = h2oai.get_dataset_split_job(boston_split_data).entity[1]
dropped = []

# Housing Experiment #1
knobs = [7, 2, 8]
housing1 = h2oai.start_experiment_sync(
      experiment_name = "Housing"
    , dataset_key = train_key
    , testset_key = test_key
    , target_col = target
    , is_classification = False
    , accuracy = knobs[0]
    , time = knobs[1]
    , interpretability = knobs[2]
    , scorer = 'RMSE'
    , enable_gpus = True
    , cols_to_drop = dropped
)

# Housing Experiment #2
knobs = [4, 2, 8]
housing2 = h2oai.start_experiment_sync(
      experiment_name = "Housing Quick"
    , dataset_key = train_key
    , testset_key = test_key
    , target_col = target
    , is_classification = False
    , accuracy = knobs[0]
    , time = knobs[1]
    , interpretability = knobs[2]
    , scorer = 'RMSE'
    , enable_gpus = True
    , cols_to_drop = dropped
)

## Forensic Glass

## Credit Card Default

In [ ]:
# Card Default: 6 4 6
# Card Monotonic: 6 4 7
# Card Big: 8 6 7
# Card GLM: 6 4 7
# Card Compliant: 6 4 7

In [ ]:
# let Driverless suggest parameters for experiment
params = h2oai.get_experiment_tuning_suggestion(
    dataset_key = train_key
    , target_col = target
    , is_classification = True
    , is_time_series = False
    , config_overrides = None)

dropped = ['name2', 'cabin', 'embarked', 'boat', 'body', 'home.dest']
knobs = [8, 2, 8]

experiment = h2oai.start_experiment_sync(
    dataset_key = train_key
    , testset_key = test_key
    , target_col = target
    , is_classification = True
    , accuracy = knobs[0]
    , time = knobs[1]
    , interpretability = knobs[2]
    , scorer = 'AUC'
    , enable_gpus = True
    , cols_to_drop = dropped
)